In [12]:
# %pip install 'vanna[chromadb,ollama]'
# %pip install 'vanna[chromadb,openai]'

In [13]:
from vanna.ollama import Ollama
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore

In [14]:
import os 
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")


In [15]:

class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key': OPENAI_API_KEY, 'model': 'gpt-4o'})


In [ ]:
# vn.connect_to_mssql(odbc_conn_str='DRIVER={ODBC Driver 17 for SQL Server};SERVER=myserver;DATABASE=mydatabase;UID=myuser;PWD=mypassword') # You can use the ODBC connection string here

In [18]:

# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

Train on Information Schema: Sparkily.stg stg_log_data
Train on Information Schema: Sparkily.stg stg_songs
Train on Information Schema: Sparkily.dw dim_time
Train on Information Schema: Sparkily.dw dim_users
Train on Information Schema: Sparkily.dw dim_artists
Train on Information Schema: Sparkily.dw dim_songs
Train on Information Schema: Sparkily.dw fact_songplays
Train on Information Schema: Sparkily.dev log_data
Train on Information Schema: Sparkily.dev song_data

In [19]:
# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)

C:\Users\tabdu\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 25.9MiB/s]


In [20]:
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS dw.dim_users (
        user_id int NULL,
        first_name varchar(16) NULL,
        last_name varchar(16) NULL,
        gender varchar(16) NULL,
        level varchar(16) NULL
    )
""")


Adding ddl: 
    CREATE TABLE IF NOT EXISTS dw.dim_users (
        user_id int NULL,
        first_name varchar(16) NULL,
        last_name varchar(16) NULL,
        gender varchar(16) NULL,
        level varchar(16) NULL
    )



'4d23a515-27cf-5d89-a91e-e42987545fa2-ddl'

In [21]:
vn.train(sql="""
SELECT 
    f.user_agent,
    u.first_name,
    u.last_name,
    s.title,
    s.duration,
    a.name AS artist_name
FROM fact_songplays f
JOIN dim_users u ON f.user_id = u.user_id
JOIN dim_songs s ON f.song_id = s.song_id
JOIN dim_artists a ON f.artist_id = a.artist_id)
WHERE f.start_time >= '2018-01-01' AND f.start_time < '2019-01-01'
""")


Using model gpt-4o for 138.25 tokens (approx)
Question generated with sql: What are the details of songs played in the year 2018, including user information and artist names? 
Adding SQL...


'c9ec6759-d39f-5c10-9e91-0c9763c3245f-sql'

In [22]:
vn.train(
    question="which users are listening to what songs, along with artist and song details.?", 
    sql="""
    SELECT 
        sp.user_agent,
        u.first_name,
        u.last_name,
        s.title,
        s.duration,
        a.artist_name
    FROM dw.fact_songplays AS sp
    JOIN dw.dim_users AS u 
        ON sp.user_id = u.user_id
    JOIN dw.dim_songs AS s 
        ON sp.song_id = s.song_id
    JOIN dw.dim_artists AS a 
        ON sp.artist_id = a.artist_id
    """
)

'd1be51ab-0808-5204-98ce-70b5e8a278e0-sql'

In [23]:
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,c9ec6759-d39f-5c10-9e91-0c9763c3245f-sql,What are the details of songs played in the ye...,"\nSELECT \n f.user_agent,\n u.first_name...",sql
1,d1be51ab-0808-5204-98ce-70b5e8a278e0-sql,"which users are listening to what songs, along...","\n SELECT \n sp.user_agent,\n ...",sql
0,4d23a515-27cf-5d89-a91e-e42987545fa2-ddl,None,\n CREATE TABLE IF NOT EXISTS dw.dim_users ...,ddl
0,aa64cfbc-d5d5-51e7-a9e5-cb41698332a6-doc,None,The following columns are in the stg_log_data ...,documentation
1,f2fccae0-7eda-573c-96a4-eea862b76da6-doc,None,The following columns are in the stg_songs tab...,documentation
2,5a5b8154-4f3e-53a1-ad70-78b1c4f465c5-doc,None,The following columns are in the dim_time tabl...,documentation
3,65d88c08-019a-595a-86f1-b5edaf580f37-doc,None,The following columns are in the dim_users tab...,documentation
4,fd3f41b0-94b8-5246-971b-b35223bfa833-doc,None,The following columns are in the dim_artists t...,documentation
5,aa53e47d-9b07-574e-a790-7f670f801d41-doc,None,The following columns are in the dim_songs tab...,documentation
6,931aac36-0a98-5100-887a-a337a2e0cfeb-doc,None,The following columns are in the fact_songplay...,documentation


In [24]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 10 is greater than number of elements in index 9, updating n_results = 9


Using model gpt-4o for 3037.75 tokens (approx)
